<a href="https://colab.research.google.com/github/jakobmwang/skillextraction/blob/main/preprocessing/translate_leftover_skill_sentences_from_gpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# install openai library
!pip install openai -qq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.7/336.7 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.1 MB/s eta 0:00:00


In [ ]:
# dependencies
import os
import re
import time
import numpy as np
import pandas as pd
import asyncio
import nest_asyncio
from openai import AsyncOpenAI
from google.colab import drive, userdata

# async fix for notebook
nest_asyncio.apply()

# file management
drive.mount('/content/drive')
WORK_DIR = '/content/drive/MyDrive/Projects/skillextraction'

# work dir shortcut function
def work_dir(*args):
    return os.path.join(WORK_DIR, *args)

Mounted at /content/drive


In [ ]:
# openai client for async
client = AsyncOpenAI(api_key=userdata.get('OpenAI'))

In [ ]:
# prompt to completion (api response)
async def get_completion(prompt):
    response = await client.chat.completions.create(
        model='gpt-4o-mini',

        messages=[{'role': 'user', 'content': prompt}]
    )
    return response

# prompts to completions (api responses)
async def get_completions(prompts):
    tasks = [get_completion(prompt) for prompt in prompts]
    return await asyncio.gather(*tasks)

In [ ]:
# prompt template
prompt = 'Please translate the following 10 sentences into Danish, and output them in plain text in the same order as they are given to you, separated by linebreak:'

In [ ]:
# get english sentence dataset
df = pd.read_csv(work_dir('Data', 'dataset.csv'))
print(df.shape)
df.head(3)

(138260, 2)


,sentence,skill
0,the ideal candidate for this position should b...,advise customers on sewing patterns
1,we need an employee who is able to assist our ...,advise customers on sewing patterns
2,if you possess good communication skills and h...,advise customers on sewing patterns


In [ ]:
# FILTERING - ONLY DO THOSE THAT DIDN'T HIT 10 FOR SOME REASON
skill_filter = [
    'apply rubber patches',
    'network marketing',
    'carry out waste water treatment',
    'carry out training in environmental matters',
    'manage research data',
    'identify fitted source for heat pumps',
    'match lyrics to mood of melody',
    'pattern grading',
    'administer hydrotherapy',
    'design customised maps',
    'quality criteria for livestock feed production',
    'merchandising techniques',
    'coordinate dock operations',
    'design floor',
    'process orders from online shop',
    'evaluate engine performance',
    'occupational science',
    'develop contingency plans for emergencies',
    'apply territory planning',
    'develop appropriate health and safety measures in accordance with available resources',
    'enter receipt of coffee inventory',
    'Wireshark',
    'plan mine operations',
    'compare shipment contents with waybill',
    'assist clients with personal development',
    'curdling of cheese',
    'operate collating machine',
    'abrasive machining processes'
]

# filter
df = df[df['skill'].isin(skill_filter)]

# check
print(len(skill_filter))
print(df.shape)
df.head(3)

28
(280, 2)


,sentence,skill
4420,responsibilities will include the operation of...,carry out waste water treatment
4421,"waste water treatment operators needed, must b...",carry out waste water treatment
4422,previous experience working with biological an...,carry out waste water treatment


In [ ]:
# generate prompt for each skill
prompts = df.groupby('skill')['sentence'].agg('\n'.join).reset_index()
prompts['sentence'] = prompt + '\n\n' + prompts['sentence']

# check example
print(prompts.shape)
prompts.head(3)

(28, 2)


,skill,sentence
0,Wireshark,Please translate the following 10 sentences in...
1,abrasive machining processes,Please translate the following 10 sentences in...
2,administer hydrotherapy,Please translate the following 10 sentences in...


In [ ]:
# check full prompt example
print(prompts.head(1)['sentence'].values[0])

Please translate the following 10 sentences into Danish, and output them in plain text in the same order as they are given to you, separated by linebreak:

desire to hire a seasoned individual with experience in using Wireshark to identify network security threats.
only candidates with a thorough understanding of Wireshark and its functions will be considered for this position.
As a network security analyst, you will utilise a variety of tools, including Wireshark, to detect and remedy any system vulnerabilities.
We are seeking a network engineer with a solid background in Wireshark and other network analysis tools to join our team.
seeking an experienced Wireshark user to join our cybersecurity team.
We require strong knowledge of Wireshark and other network analysis tools to identify and address any security vulnerabilities.
Wireshark experience is a must-have for this network security position.
our ideal candidate should have experience working with network protocols and analysing t

In [ ]:
# separate in chunks of 100
chunks = np.array_split(prompts, np.arange(100, len(prompts), 100))

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [ ]:
# get completions for each chunk
for i, chunk in enumerate(chunks):
    filename = os.path.join(WORK_DIR, 'Translated_data', 'leftover_translated_sentences_{}.csv'.format(i))
    if os.path.exists(filename):
        continue
    completions = await get_completions(chunk['sentence'].values)
    completions = [[t for s in c.choices[0].message.content.split('\n') if (t := re.sub(r'^[\W0-9]+', '', s).strip()) != ''] for c in completions]
    chunk['sentence'] = completions
    chunk.explode('sentence')[['skill', 'sentence']].to_csv(filename, index=False)
    time.sleep(50) # avoiding rate limits

In [ ]:
# sanity check
df = pd.read_csv(os.path.join(WORK_DIR, 'Translated_data', 'leftover_translated_sentences_{}.csv'.format(i)))
df['skill'].value_counts()

skill
Wireshark                                                                                10
abrasive machining processes                                                             10
process orders from online shop                                                          10
plan mine operations                                                                     10
pattern grading                                                                          10
operate collating machine                                                                10
occupational science                                                                     10
network marketing                                                                        10
merchandising techniques                                                                 10
match lyrics to mood of melody                                                           10
manage research data                                                      

In [ ]:
# check missing skill (!)
df[df['skill'] == 'quality criteria for livestock feed production']['sentence'].tolist()

['Den ideelle kandidat skal værdsætte nøjagtighed og præcision i kvalitetssikringsprocessen for foderproduktion, som overholder en streng serie af kvalitetskriterier og procedurer for at sikre en konsistent produktion af højkvalitets husdyrfoder.',
 'Vi søger en person med viden om den ernæringsmæssige værdi af husdyrfoder og erfaring med laboratorievurdering.',
 'Forberede og gennemføre gennemførlighedsstudier for at evaluere ydeevnen af laboratoriet til analyse af husdyrfoder.',
 'Erfaring med kvalitetssikring og analyse af husdyrfoder er påkrævet til denne stilling.',
 'Skal kunne udføre laboratorieanalyser og nøjagtigt bestemme den ernæringsmæssige værdi af foderingredienser.',
 'Ansøgeren bør have en stærk forståelse for kvalitetskriterier for husdyrfoderproduktion.',
 'Erfaring med at evaluere foderanalyse laboratorier og sikre overholdelse af internationale standarder foretrækkes.',
 'En succesfuld kandidat til dette job skal være fortrolig med kvalitetskriterier, der er i overe

In [ ]:
# manual fix (from web gpt)
df.loc[len(df)] = {'skill': 'quality criteria for livestock feed production',
                   'sentence': 'Du værdsætter præcision og overholder strenge kvalitetsstandarder for at sikre en konsistent produktion af højkvalitets husdyrfoder.'}

# check
df['skill'].value_counts()

skill
Wireshark                                                                                10
abrasive machining processes                                                             10
process orders from online shop                                                          10
plan mine operations                                                                     10
pattern grading                                                                          10
operate collating machine                                                                10
occupational science                                                                     10
network marketing                                                                        10
merchandising techniques                                                                 10
match lyrics to mood of melody                                                           10
manage research data                                                      

In [ ]:
# save!
df.to_csv(os.path.join(WORK_DIR, 'Translated_data', 'leftover_translated_sentences_{}.csv'.format(i)), index=False)